In [2]:
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
from dask.diagnostics import ProgressBar as pb
import seaborn as sns

import pickle

In [23]:
# import data
df = dd.read_csv('processed.csv')
df = df.drop('Unnamed: 0', axis=1)
df.time = dd.to_datetime(df['time'])
df = df[['gid','action']]

# sort out buy action
df1 =  df.copy()
df1 = df1.loc[df1.action=='buy']
df1 = df1.groupby('gid').count()
df1 = df1.sort_values('action',ascending=False)

# get top ids
def get_top(n=20, action='pv'):
    df1 =  df.copy()
    df1 = df1.loc[df1.action==action]
    df1 = df1.groupby('gid').count()
    df1 = df1.sort_values('action',ascending=False)
    return df1.head(20)

In [32]:
top_buy = get_top(action='buy')
top_pv = get_top(action='pv')
top_cart = get_top(action='cart')
top_fav = get_top(action='fav')

In [24]:
# save data
import pickle

# with open('top_buy.pkl','wb') as file:
#     pickle.dump(top_buy, file)
    
# with open('top_pv.pkl','wb') as file:
#     pickle.dump(top_pv, file)
    
# with open('top_cart.pkl','wb') as file:
#     pickle.dump(top_cart, file)
    
# with open('top_fav.pkl','wb') as file:
#     pickle.dump(top_fav, file)
    
with open('top_buy.pkl','rb') as file:
    top_buy = pickle.load(file)
    
with open('top_pv.pkl','rb') as file:
    top_pv = pickle.load(file)
    
with open('top_cart.pkl','rb') as file:
    top_cart = pickle.load(file)
    
with open('top_fav.pkl','rb') as file:
    top_fav = pickle.load(file)


In [29]:
# fav and buy
set(top_fav.index)&set(top_buy.index)

{740947, 1535294, 3031354}

In [28]:
# cart and buy
set(top_cart.index)&set(top_buy.index)

{705557, 1535294, 2560262, 3031354, 4443059}

In [27]:
# pv and buy
set(top_pv.index)&set(top_buy.index)

{1535294, 3031354, 4443059}

In [26]:
# pv and fav, pv and cart
set(top_pv.index)&set(top_fav.index),set(top_pv.index)&set(top_cart.index),

({138964,
  812879,
  1535294,
  1583704,
  2032668,
  2279428,
  2331370,
  2453685,
  2818406,
  3031354,
  3845720,
  4649427},
 {138964,
  812879,
  1535294,
  1583704,
  2279428,
  2331370,
  2453685,
  2818406,
  3031354,
  3845720,
  4211339,
  4443059})

1. 可以看出，三种前置行为中，加购和购买行为的联系是最紧密的，前二十名中有5个相同。而浏览和收藏差不多，都只有3个与前20销量相同，这说明收藏和购买的关系比较弱。
2. 此外，分析浏览行为和收藏、加购的关系，可以发现这三个名单的重合率较高，收藏和加购前20各有12个在最高销售榜单里。这说明浏览量多的商品更容易被收入购物车或是收藏，但结合前面的关系，不是所有商品都会被购买，且进入购物车被购买的可能性更大。

In [22]:
df = dd.read_csv('processed.csv')
df = df.drop('Unnamed: 0', axis=1)
df.time = dd.to_datetime(df['time'])
df = df[['uid','action','time','cid']]
og = df.loc[df.action=='buy']
og = og.compute()

In [7]:


with open('og.pkl','wb') as file:
    pickle.dump(og, file)
with open('og.pkl','rb') as file:
    og = pickle.load(file)


In [23]:
# top 20 cats
top_cat = og.groupby('cid').count().sort_values('action',ascending=False)['action'].head(20)
top_cat

cid
1464116    34591
2735466    33730
2885642    31844
4145813    31658
4756105    28258
4801426    26495
982926     24825
2640118    18332
4159072    18016
1320293    17137
3002561    16472
4357323    15854
4789432    15696
903809     15447
4217906    14442
1299190    13723
570735     13711
2465336    13350
1879194    12868
3607361    12721
Name: action, dtype: int64

In [30]:
# R & F
F_list = og.groupby('uid').count().sort_values('action',ascending=False)['action']
R_list = og.groupby('uid').max().sort_values('time',ascending=False)['time']
F_list,\
R_list

(uid
 486458     262
 866670     177
 702034     159
 107013     131
 1014116    118
           ... 
 505702       1
 505699       1
 505698       1
 505697       1
 1018011      1
 Name: action, Length: 672404, dtype: int64,
 uid
 333794   2017-12-03 15:59:59
 189864   2017-12-03 15:59:59
 871420   2017-12-03 15:59:59
 461596   2017-12-03 15:59:59
 206291   2017-12-03 15:59:59
                  ...        
 128861   2017-11-24 16:00:13
 531029   2017-11-24 16:00:08
 38184    2017-11-24 16:00:04
 39168    2017-11-24 16:00:00
 73805    2017-11-24 16:00:00
 Name: time, Length: 672404, dtype: datetime64[ns])

In [71]:
# standardize and add up
ndf = dd.concat([F_list,R_list],axis=1)
ndf = ndf.compute()
from sklearn import preprocessing


ndf['F'] = preprocessing.scale(ndf.action)
ndf['R'] = preprocessing.scale((ndf.time - pd.Timestamp(2017,12,4,0)).astype('timedelta64[h]').view('float'))
ndf['RF score'] = ndf['R'] + ndf['F']
ndf.sort_values(by='RF score', inplace=True,ascending=False)
ndf

,action,time,F,R,RF score
uid,,,,,
486458,262,2017-12-03 13:27:24,89.473433,1.152044,90.625477
866670,177,2017-12-02 12:44:08,60.109797,0.723909,60.833706
702034,159,2017-12-03 14:57:57,53.891615,1.169169,55.060784
107013,131,2017-12-03 13:21:33,44.218887,1.152044,45.370931
1014116,118,2017-12-03 14:01:17,39.727978,1.169169,40.897147
...,...,...,...,...,...
739218,1,2017-11-24 16:20:16,-0.690204,-2.495660,-3.185864
958505,1,2017-11-24 16:09:15,-0.690204,-2.495660,-3.185864
388388,1,2017-11-24 16:47:36,-0.690204,-2.495660,-3.185864


In [ ]:
nums = [-1,0,3,5,9,12]
target = 2


In [ ]:
left = 0
right = len(nums)-1
mid = (left+right)//2
while nums[mid] != target:
    if target < nums[left]:
        left = left - (right-left)
        right = left            
    elif target > nums[right]:
        right = right + (right - left)
        left = right
    else:
        left = (right+left)//2
        mid = (left+right)//2


In [1]:
1 // 2

0